# Eukaryote screened functional analysis

We are being plagued by eukaryotic GO terms infesting enrichment results, so the simplest solution seems to be to filter out any sequences of eukaryotic origin by mapping all the protein sequences against uniprot and filter out those which hit eukaryotes. Interproscan used 1000 chunks which are going to be too small, so firstly rechunk the input to 100 files.

In [6]:
import pandas as pd
import numpy as np
import os
import re
import pickle
from itertools import chain
from IPython.display import display, Markdown,HTML

samples=('2000', '2001', '2002', '2006', '2007', '2009', '2011', '2012', '2013', '2023', '2024', '2025')
contrast_list=( 'Bulk_Desert','Bulk_North', 'Bulk_Elite', 'Desert_Elite', 'North_Elite', 'Desert_North' )
pd.set_option('display.max_rows',None)

In [ ]:
%%bash
mkdir -p fasta
../bin/chunk_fasta.py --fasta ../mmseqs_clusters/mmseqs_rep_seq.fasta --outdir fasta --chunks 100

## Eukaryotic protein identification

The processing itself is done with a snakemake workflow, which runs the following rules.

Please place the mmseqs protein prediction fasta files in a `fasta` subdirectory of this directory prior to running the workflow with `run_workflow.sh`. As with previous scripts/workflows this may need submission parameters adjusting to match your local setup, both in `run_workflow.sh` and `config.json`.

* taxonomy: Downloads the NCBI taxonomy database and parses the nodes.dmp file and pickles the resulting structure.
* uniprot: Downloads uniprot and builds diamond index
* diamond:  Runs diamond against the uniprot database (release 2021_02) producing a tab-delimited output with the default of up to 25 hits per sequence. Just reporting one hit would not necessarily report the best hit...
* euk_find: Takes the top hit for each query and parses it's taxid from the description line, then walks up the taxonomy tree to find the superkingdom, reporting ids of proteins which are eukaryotes
* merge: Combines per-chunk eukaryote lists into single file (`eukaryotic_protein_ids.txt`) containing the ids of all identified eukaryotic proteins


In [29]:
%%bash
wc -l eukaryotic_protein_ids.txt

439249 eukaryotic_protein_ids.txt


We seem to have 439249 eukaryotic proteins out of a total of 10554104 (4.16%)

# Generate read counts from cDNA alignments

Reuse previous get_counts function

In [30]:
def get_counts(align_dir: str, sample:str):
    
    """
    Obtains counts of properly paired and singleton reads for a sample from samtools idxstats output
    
    Required parameters:
    align_dir (str): path to directory containing idxstats outputs
    sample (str): sample name
    
    Returns:
        counts(pd.DataFrame): Pandas dataframe of counts
    """
    
    colnames=('protein_id','length','paired_count','whoknows')
    proper_pairs_counts=pd.read_csv(
        '{}/{}_mmseqs_rep_cDNA_seq.proper.idxstats'.format(align_dir,sample), sep="\t",names=colnames)
    proper_pairs_counts=proper_pairs_counts[['protein_id','paired_count']]
    
    colnames=('protein_id','length','single_count','whoknows')
    singleton_counts=pd.read_csv('{}/{}_mmseqs_rep_cDNA_seq.singletons.idxstats'.format(align_dir,sample),sep="\t",names=colnames)
    singleton_counts=singleton_counts[['protein_id','single_count']]
    
    counts=pd.merge(proper_pairs_counts,singleton_counts,how='inner',on='protein_id')
    counts[sample]=counts['paired_count']+counts['single_count']
    counts=counts[['protein_id',sample]]
    
    return counts

InterProScan has produced assignments for 1530296 proteins, which are contained in the `interproscan/iprscan_parsed.txt` file. We can create an output file suitable for enrichment analysis by combining this with per-protein abundances from the cDNA alignments.

In [ ]:
cdna_dir='../functional_enrichment/cDNA_alignments/'
ipr_dir='../functional_enrichment/interproscan/'

go_mappings=pd.read_csv('{}/iprscan_parsed.txt'.format(ipr_dir), sep="\t", header=0,names=('protein_id','GOs'))
all_counts=go_mappings['protein_id']
for sample in samples:
    counts=get_counts(cdna_dir,sample)
    all_counts=pd.merge(all_counts,counts,how='inner',on='protein_id')

merged_counts=pd.merge(all_counts,go_mappings,how='inner',on='protein_id')
merged_counts=merged_counts[['protein_id','GOs','2000','2001','2002','2006','2007','2009','2011','2012','2013','2023','2024','2025']]
merged_counts=merged_counts.dropna()
merged_counts.to_csv('cdna_proper_count_summary.txt',sep='\t',index=False)

Counts for invalid pairs were produced with a modified version of the previous `count_mispaired_cdna.py` script, tweaked to cope with the interproscan results.

In [ ]:
%%bash
qsub -t 1-12 ./eukaryote_filter/bin/count_mispaired_cdna.py --bam_dir ../cDNA_alignments/ --out_dir unpaired_counts --ipr ../interproscan/iprscan_parsed.txt

The per-sample invalid pairs counts results need to be combined...

In [ ]:
invalid_pairs_dir='unpaired_counts/'

invalid_pairs=pd.DataFrame()
for sample in samples:
    if os.path.isfile('{}/{}_mmseqs_rep_cDNA_seq.invalid_pairs.read_count_summary.txt'.format(invalid_pairs_dir,sample)):
        sample_df=pd.read_csv('{}/{}_mmseqs_rep_cDNA_seq.invalid_pairs.read_count_summary.txt'.format(invalid_pairs_dir, sample), sep="\t",index_col=0)
    else:
        sample_df=pd.DataFrame(columns=[sample])
    invalid_pairs=pd.merge(invalid_pairs,sample_df,how='outer',left_index=True, right_index=True)
    
invalid_pairs.fillna('0',inplace=True)
colnames=invalid_pairs.columns
colnames=list(map(lambda x: x.replace('_mmseqs_rep_cDNA_seq',''),colnames))
invalid_pairs.columns=colnames
invalid_pairs['protein_id']=invalid_pairs.index
invalid_pairs.to_csv('cdna_invalid_pairs_count_summary'.format(invalid_pairs_dir), sep="\t", index=False)

Reset counts columns to floats to ensure they are consistent and can be merged, then merge the dataframes and produce summed counts for each sample

In [ ]:
proper_counts=merged_counts.astype({'2000':float,'2001':float,'2002':float,'2006':float,'2007':float,'2009':float,
                                    '2011':float,'2012':float,'2013':float,'2023':float,'2024':float,'2025':float})

invalid_pairs_counts=invalid_pairs.astype({'2000':float,'2001':float,'2002':float,'2006':float,'2007':float,'2009':float,
                                           '2011':float,'2012':float, '2013':float,'2023':float,'2024':float,'2025':float})

combined_counts=proper_counts.merge(invalid_pairs_counts,how='outer',on='protein_id')
combined_counts.fillna(value=0,inplace=True)

for sample in samples:
    col1='{}_x'.format(sample)
    col2='{}_y'.format(sample)
    combined_counts[sample]=combined_counts[col1]+combined_counts[col2]
    combined_counts.drop([col1,col2],inplace=True,axis=1)
    
with open('eukaryotic_protein_ids.txt') as fh:
    euk_ids=fh.read().splitlines()
    
print('Total proteins: ',len(combined_counts))
# drop proteins identified as being of eukaryotic origin
combined_counts=combined_counts[~combined_counts.protein_id.isin(euk_ids)]
print('Non-eukaryotic proteins:',len(combined_counts))
combined_counts.to_csv('full_cdna_count_summary.txt'.format(dir),sep='\t',index=False)

There still appear to be some eukaryotic terms, however the numbers are greatly reduced. Those which remain look to be hitting taxon 070528, which is a metagenome, so we can't exclude these. Now use count_go_terms.py to convert per-cdna counts into per-go term counts (`go_count_table.txt`).

In [ ]:
%%bash
qsub ../functional_enrichment/interproscan/bin/count_go_terms.py --dir .

## Slim mapping

In [ ]:
go_dir='GO_slim_mapping/'
os.mkdir(go_dir)

terms_df=pd.read_csv('go_count_table.txt',sep='\t',index_col=0)
terms=set(terms_df.index.values.tolist())
terms=list(terms)

# Subset terms to include only those from the biological_process namespace
go=pd.read_csv('../GO_slim_mapping/go_table.txt',sep="\t",header=None)
biol_proc=go[go[1]=='biological_process']
biol_proc_terms=biol_proc[biol_proc[0].isin(terms)]
terms=biol_proc_terms[0].tolist()

gaf_file=open('{}/go_terms.gaf'.format(go_dir),'w')
gaf_file.write('!gaf-version: 2.0\n')
count=1
for term in terms:
    gaf_file.write('WB\t{}\t{}\toptional\t{}\tpubmed\tunknown\toptional\tunknown\toptional\toptional\tprotein\tunknown\t{}\tWB\toptional\toptional\n'.format(count,count,term,count))
    count+=1
gaf_file.close()

The paths to the go-basic.obo and goslim_terms.txt files may need adjusting in the following cell.

NB: owltools is not available in conda, so needs downloading separately from https://github.com/owlcollab/owltools

In [ ]:
%%bash
cd GO_slim_mapping
ln -s ../../../../GO_data/go-basic.obo .
ln -s ../../GO_slim_mapping/goslim_terms.txt .

owltools go-basic.obo --gaf go_terms.gaf --map2slim --idfile goslim_terms.txt --write-gaf go_slim_mapped_terms.gaf

In [ ]:
orig_terms=pd.read_csv('{}/go_terms.gaf'.format(go_dir),sep="\t",header=None,skiprows=1)
slim_terms=pd.read_csv('{}/go_slim_mapped_terms.gaf'.format(go_dir),sep="\t",header=None,skiprows=5)
orig_terms=orig_terms.iloc[:,[1,4]]
slim_terms=slim_terms.iloc[:,[1,4]]
slim_mapping=pd.merge(orig_terms,slim_terms,how='right',on=1)
slim_mapping=slim_mapping.iloc[:,[1,2]]
slim_mapping.columns=['GO','Slim']
# create a subset of go terms which are _only_ those which map to the slim for downstream analysis
mapped_terms=slim_mapping['GO'].tolist()
with open ('{}/mapped_go_terms.txt'.format(go_dir),'w') as fh:
    fh.write("\n".join(list(set(mapped_terms))))

Now get the counts of the slim terms...

In [ ]:
%%bash
qsub ../bin/count_go_slim_terms.py --goterm_gaf GO_slim_mapping/go_terms.gaf --slim_gaf GO_slim_mapping/go_slim_mapped_terms.gaf --counts full_cdna_count_summary.txt --out GO_slim_mapping/slim_counts.txt

Subsequent analysis was carried out in the `MAG_eukaryote_screened_GO_enrichment.Rmd` notebook. PCAs of the rld normalised GO terms and slims:
![GO Slims PCA plot](GO_Slims_pca.png)

In [10]:
slim_terms=pd.read_csv('../analysis_results/eukaryote_screened/top20_slim_terms.txt',sep="\t")
pd.set_option('display.precision',2)

for contrast in contrast_list:
    contrast_df=pd.read_csv('../analysis_results/eukaryote_screened/slim_{}.txt'.format(contrast),sep="\t")
    contrast_df=contrast_df[['GO','log2FoldChange','padj']]
    contrast_df.columns=('GO','{} l2fc'.format(contrast),'{} padj'.format(contrast))
    slim_terms=pd.merge(slim_terms,contrast_df,how='left',on='GO')
slim_terms.replace(np.nan,'',regex=True,inplace=True)

display(slim_terms)

GO                                              name  Dim.1  \
0   GO:0000160           phosphorelay signal transduction system   2.72   
1   GO:0005975                    carbohydrate metabolic process  17.74   
2   GO:0006259                             DNA metabolic process   2.25   
3   GO:0006508                                       proteolysis   0.44   
4   GO:0006629                           lipid metabolic process   2.48   
5   GO:0006950                                response to stress   0.53   
6   GO:0007155                                     cell adhesion   0.65   
7   GO:0009405                                      pathogenesis  17.30   
8   GO:0009628                      response to abiotic stimulus   0.94   
9   GO:0015979                                    photosynthesis   0.88   
10  GO:0016070                             RNA metabolic process   1.22   
11  GO:0016226                      iron-sulfur cluster assembly   0.74   
12  GO:0016310                                   phosphorylation   6.53   
13  GO:0017000                   antibiotic biosynthetic process   0.34   
14  GO:0019222                   regulation of metabolic process   1.57   
15  GO:0042221                              response to chemical  33.36   
16  GO:0043934                                       sporulation   0.82   
17  GO:0065003               protein-containing complex assembly   0.39   
18  GO:0071840     cellular component organization or biogenesis   2.99   
19  GO:0071973  bacterial-type flagellum-dependent cell motility   4.16   

       Dim.2  Bulk_Desert l2fc  Bulk_Desert padj  Bulk_North l2fc  \
0   1.18e+00              0.10          3.20e-02             0.12   
1   6.79e+00              0.23          3.59e-07             0.29   
2   2.47e+01              0.02          6.88e-01            -0.02   
3   1.22e+01             -0.08          5.38e-02            -0.07   
4   1.94e-02              0.12          4.61e-08             0.19   
5   4.09e-01             -0.05          6.14e-03            -0.07   
6   2.30e+01             -0.06          6.62e-01             0.03   
7   7.16e+00              1.21          4.64e-05             1.25   
8   3.05e-04              1.33          4.65e-04             1.45   
9   1.20e-01             -0.84          1.18e-04            -0.89   
10  1.94e+00              0.05          1.48e-01             0.04   
11  2.61e-04             -0.16          7.53e-06            -0.16   
12  1.27e+01             -0.14          3.24e-02            -0.18   
13  2.31e+00             -0.24          2.72e-03            -0.15   
14  4.99e-03             -0.21          2.72e-03            -0.21   
15  1.01e-01              0.61          7.40e-11             0.61   
16  3.01e-05             -0.48          2.08e-05            -0.57   
17  7.88e-01              0.59          4.67e-07             0.60   
18  2.02e+00              0.12          1.90e-02             0.18   
19  4.86e-01              0.75          3.82e-13             0.71   

    Bulk_North padj  Bulk_Elite l2fc  Bulk_Elite padj  Desert_Elite l2fc  \
0          1.24e-02             0.14         1.58e-03           3.98e-02   
1          2.42e-11             0.34         1.03e-14           1.05e-01   
2          6.97e-01            -0.16         8.53e-05          -1.81e-01   
3          5.79e-02            -0.02         7.04e-01           6.06e-02   
4          5.10e-20             0.20         6.54e-22           8.21e-02   
5          1.33e-04            -0.06         3.04e-03          -2.65e-03   
6          8.11e-01             0.26         2.17e-02           3.23e-01   
7          2.65e-05             1.80         2.66e-10           5.88e-01   
8          1.03e-04             1.70         3.20e-06           3.66e-01   
9          4.13e-05            -1.20         1.03e-08          -3.60e-01   
10         2.23e-01             0.13         7.18e-05           7.58e-02   
11         1.55e-06            -0.21         1.70e-10          -5.77e-02  

## Count distribution for top 10 Slim terms

The figure was created in the `MAG_eukaryote_screened_GO_enrichment.Rmd` notebook, with the terms ordered starting from those contributing most to bulk, them moving rount to the wild lines before finishing with elite.

![Top 20 slim term count distributions](slim_term_counts_euk_screened_20.png)

and for the Full GO BP ontology...

![GO term PCA plot](GO_Terms_pca.png)

Below are the results of DESeq2 analysis for these terms, along with their description. Missing values indicate there was no significant change identified (adjust p-value <0.05)

In [54]:
go_terms=pd.read_csv('../analysis_results/eukaryote_screened/top20_go_terms.txt',sep="\t",names=('GO','name'),skiprows=1)

for contrast in contrast_list:
    contrast_df=pd.read_csv('../analysis_results/eukaryote_screened/go_{}.txt'.format(contrast),sep="\t")
    contrast_df=contrast_df[['GO','log2FoldChange','padj']]
    contrast_df.columns=('GO','{} l2fc'.format(contrast),'{} padj'.format(contrast))
    go_terms=pd.merge(go_terms,contrast_df,how='left',on='GO')
go_terms.replace(np.nan,'',regex=True,inplace=True)
pd.set_option('display.precision',3)
display(go_terms)

GO                                               name  \
0   GO:0000302                response to reactive oxygen species   
1   GO:0006203                             dGTP catabolic process   
2   GO:0006683               galactosylceramide catabolic process   
3   GO:0006935                                         chemotaxis   
4   GO:0009255  Entner-Doudoroff pathway through 6-phosphogluc...   
5   GO:0009405                                       pathogenesis   
6   GO:0009584                         detection of visible light   
7   GO:0009847                                  spore germination   
8   GO:0015891                              siderophore transport   
9   GO:0015979                                     photosynthesis   
10  GO:0015995                   chlorophyll biosynthetic process   
11  GO:0019290                   siderophore biosynthetic process   
12  GO:0019629  propionate catabolic process, 2-methylcitrate ...   
13  GO:0042128                               nitrate assimilation   
14  GO:0042545                             cell wall modification   
15  GO:0043213                              bacteriocin transport   
16  GO:0044209                                        AMP salvage   
17  GO:0045893  positive regulation of transcription, DNA-temp...   
18  GO:0046373                      L-arabinose metabolic process   
19  GO:0051156              glucose 6-phosphate metabolic process   

   Bulk_Desert l2fc Bulk_Desert padj  Bulk_North l2fc  Bulk_North padj  \
0              1.86         6.18e-05            2.362        1.588e-07   
1              1.66         9.96e-06            1.940        1.279e-07   
2              2.04          5.3e-07            2.531        1.694e-10   
3             0.883         5.43e-13            0.883        3.852e-13   
4              1.35          1.3e-08            1.199        3.722e-07   
5              1.17          0.00103            1.230        4.103e-04   
6               1.6          0.00061            1.758        1.169e-04   
7              1.51            0.011            1.738        2.457e-03   
8             0.758         2.58e-09            0.970        9.209e-15   
9             -1.28          5.9e-05           -1.368        1.158e-05   
10            -1.44         4.61e-06           -1.592        2.362e-07   
11             1.08         3.42e-06            1.262        3.067e-08   
12             2.93         8.36e-10            3.064        8.319e-11   
13             1.86         1.82e-12            1.911        3.852e-13   
14                                              1.491        8.128e-03   
15             1.67         1.55e-05            1.806        1.633e-06   
16             1.45         3.62e-06            1.515        7.792e-07   
17             2.39         9.09e-17            2.069        1.029e-12   
18             1.32         7.76e-29            1.509        5.955e-38   
19             1.22         1.39e-08            1.306        6.491e-10   

   Bulk_Elite l2fc Bulk_Elite padj Desert_Elite l2fc Desert_Elite padj  \
0             3.06        1.34e-12               1.2            0.0298   
1             1.86        1.86e-07                                       
2             2.66        4.96e-12                                       
3             1.08        4.91e-20                                       
4              1.7        6.36e-14                                       
5             1.79        4.61e-08                                       
6             2.09        1.58e-06                                       
7                                                                        
8             1.45        1.48e-33             0.688          2.08e-06   
9             -1.8        1.79e-09                                       
10           -1.98        2.92e-11                                       
11            1.45        3.67e-11                                       
12             3.1        1.47e-11           

### Count distributions in top GO terms

The figure below was created in the `MAG_eukaryote_screened_GO_enrichment.Rmd` notebook, with the terms arranged by contribution from the PCA, starting with the strongest 'wild' term, moving round the plot to the strongest 'elite' term. The contribution arrow was added in  illustrator for convenience.

![Top 20 GO terms count distribution](go_term_counts_euk_screened_20.png)

## K-means clustering

An alternative approach is to carry out k-means clustering and look for clusters showing interesting patterns of change. Clustering of GO terms from interproscan was carried out with 10 centroids.

![GO term k-means clustering](ipr_kmeans_10.png)

Clusters 5 and 8 look interesting...below are the cluster members along with their DESeq results

In [71]:
cluster=pd.read_csv('../analysis_results/eukaryote_screened/clusters/ipr_kmeans_cluster_5.txt',sep="\t")
names=cluster.columns.map(lambda x: x.replace('log2FoldChange','l2fc'))
cluster.columns=names
display(Markdown('### Cluster 5'))
display(cluster)
cluster=pd.read_csv('../analysis_results/eukaryote_screened/clusters/ipr_kmeans_cluster_8.txt',sep="\t")
names=cluster.columns.map(lambda x: x.replace('log2FoldChange','l2fc'))
cluster.columns=names
display(Markdown('### Cluster 8'))
display(cluster)

### Cluster 5

GO  cluster                                               name  \
0   GO:0006002        5             fructose 6-phosphate metabolic process   
1   GO:0034755        5                   iron ion transmembrane transport   
2   GO:0006665        5                     sphingolipid metabolic process   
3   GO:0071555        5                             cell wall organization   
4   GO:0019684        5                     photosynthesis, light reaction   
5   GO:0018298        5                        protein-chromophore linkage   
6   GO:0009584        5                         detection of visible light   
7   GO:0006569        5                       tryptophan catabolic process   
8   GO:0006559        5                  L-phenylalanine catabolic process   
9   GO:0006570        5                         tyrosine metabolic process   
10  GO:0006741        5                          NADP biosynthetic process   
11  GO:0042819        5                    vitamin B6 biosynthetic process   
12  GO:0042823        5           pyridoxal phosphate biosynthetic process   
13  GO:0090124        5                        N-4 methylation of cytosine   
14  GO:0030435        5  sporulation resulting in formation of a cellul...   
15  GO:0009297        5                                     pilus assembly   
16  GO:0006269        5           DNA replication, synthesis of RNA primer   
17  GO:0019679        5  propionate metabolic process, methylcitrate cycle   
18  GO:0045910        5           negative regulation of DNA recombination   
19  GO:0006974        5           cellular response to DNA damage stimulus   
20  GO:0018142        5                 protein-DNA covalent cross-linking   
21  GO:0072488        5                   ammonium transmembrane transport   
22  GO:0031554        5  regulation of DNA-templated transcription, ter...   
23  GO:0044781        5              bacterial-type flagellum organization   
24  GO:0008612        5  peptidyl-lysine modification to peptidyl-hypusine   
25  GO:0019290        5                   siderophore biosynthetic process   
26  GO:0042254        5                                ribosome biogenesis   
27  GO:0000162        5                    tryptophan biosynthetic process   
28  GO:0051156        5              glucose 6-phosphate metabolic process   
29  GO:0006007        5                          glucose catabolic process   
30  GO:0009255        5  Entner-Doudoroff pathway through 6-phosphogluc...   
31  GO:0010498        5              proteasomal protein catabolic process   
32  GO:0019941        5   modification-dependent protein catabolic process   
33  GO:0070475        5                              rRNA base methylation   
34  GO:0006241        5                           CTP biosynthetic process   
35  GO:0030253        5   protein secretion by the type I secretion system   
36  GO:0030245        5                        cellulose catabolic process   
37  GO:0009263        5           deoxyribonucleotide biosynthetic process   
38  GO:0008615        5                    pyridoxine biosynthetic process   
39  GO:0030026        5                 cellular manganese ion homeostasis   
40  GO:0046688        5                             response to copper ion   
41  GO:0019674        5                              NAD metabolic process   
42  GO:0042619        5          poly-hydroxybutyrate biosynthetic process   
43  GO:0006303        5  double-strand break repair via nonhomologous e...   
44  GO:0030255        5  protein secretion by the type IV secretion system   
45  GO:0006146        5                          adenine catabolic process   
46  GO:0006064        5                      glucuronate catabolic process   
47  GO:0006729        5           tetrahydrobiopterin biosynthetic process   
48  GO:0001514        5                       selenocysteine incorporation   
49  GO:0005976        5                   polysaccharide metabolic process   
50  GO:0090529        5           

### Cluster 8

GO  cluster                                               name  \
0   GO:0044780        8                  bacterial-type flagellum assembly   
1   GO:0071586        8                        CAAX-box protein processing   
2   GO:0005992        8                     trehalose biosynthetic process   
3   GO:0006304        8                                   DNA modification   
4   GO:0000413        8              protein peptidyl-prolyl isomerization   
5   GO:0008643        8                             carbohydrate transport   
6   GO:0032784        8  regulation of DNA-templated transcription, elo...   
7   GO:0071705        8                        nitrogen compound transport   
8   GO:0071973        8   bacterial-type flagellum-dependent cell motility   
9   GO:0016539        8                   intein-mediated protein splicing   
10  GO:0006865        8                               amino acid transport   
11  GO:0015693        8                            magnesium ion transport   
12  GO:0008610        8                         lipid biosynthetic process   
13  GO:0046373        8                      L-arabinose metabolic process   
14  GO:0006799        8                 polyphosphate biosynthetic process   
15  GO:0015696        8                                 ammonium transport   
16  GO:0006428        8                      isoleucyl-tRNA aminoacylation   
17  GO:0031564        8                      transcription antitermination   
18  GO:0000270        8                    peptidoglycan metabolic process   
19  GO:0009405        8                                       pathogenesis   
20  GO:0071709        8                                  membrane assembly   
21  GO:0035435        8              phosphate ion transmembrane transport   
22  GO:0006814        8                               sodium ion transport   
23  GO:0019441        8         tryptophan catabolic process to kynurenine   
24  GO:0009103        8            lipopolysaccharide biosynthetic process   
25  GO:0006568        8                       tryptophan metabolic process   
26  GO:0009234        8                   menaquinone biosynthetic process   
27  GO:0035999        8                   tetrahydrofolate interconversion   
28  GO:0019264        8           glycine biosynthetic process from serine   
29  GO:0033567        8       DNA replication, Okazaki fragment processing   
30  GO:0006750        8                   glutathione biosynthetic process   
31  GO:0006413        8                           translational initiation   
32  GO:0006493        8                     protein O-linked glycosylation   
33  GO:0006821        8                                 chloride transport   
34  GO:0009063        8              cellular amino acid catabolic process   
35  GO:0006825        8                               copper ion transport   
36  GO:0015891        8                              siderophore transport   
37  GO:0009236        8                     cobalamin biosynthetic process   
38  GO:0001510        8                                    RNA methylation   
39  GO:0009073        8    aromatic amino acid family biosynthetic process   
40  GO:0071805        8              potassium ion transmembrane transport   
41  GO:0006275        8                      regulation of DNA replication   
42  GO:0009311        8                  oligosaccharide metabolic process   
43  GO:0009072        8       aromatic amino acid family metabolic process   
44  GO:0032508        8                               DNA duplex unwinding   
45  GO:0005978        8                      glycogen biosynthetic process   
46  GO:0000902        8                                 cell morphogenesis   
47  GO:0006433        8                         prolyl-tRNA aminoacylation   
48  GO:0006431        8                      methionyl-tRNA aminoacylation   
49  GO:0006231        8                          dTMP biosynthetic process   

    Bulk_Desert.l2fc  Bulk_Desert